IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M3\FAULT_M3_10


In [3]:
guasto=2

VOLO m3 FAULT 10%

In [4]:
rcou_m3_fault10 = pd.read_csv("RCOU.csv")
rcou_m3_fault10 = rcou_m3_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m3_fault10 = rcou_m3_fault10[((rcou_m3_fault10['C9'] >= potenza) & (rcou_m3_fault10['C10'] >= potenza) & (rcou_m3_fault10['C11'] >= 1350) & (rcou_m3_fault10['C12'] >= potenza) & (rcou_m3_fault10['C13'] >= potenza) & (rcou_m3_fault10['C14']>= potenza))]
rcou_m3_fault10=rcou_m3_fault10.reset_index(drop=True)
display(rcou_m3_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,106891806,1479,1406,1428,1459,1460,1427
1,106907540,1455,1450,1499,1401,1499,1402
2,106920441,1362,1542,1506,1409,1415,1501
3,106931177,1545,1371,1425,1502,1509,1418
4,106942020,1535,1398,1429,1510,1496,1444
...,...,...,...,...,...,...,...
6827,186281436,1556,1438,1522,1478,1504,1495
6828,186291822,1552,1431,1521,1467,1504,1485
6829,186302079,1521,1451,1535,1435,1493,1482
6830,186312025,1502,1450,1546,1397,1494,1458


In [5]:
min=rcou_m3_fault10['TimeUS'][0]
max=rcou_m3_fault10['TimeUS'][len(rcou_m3_fault10)-1]
print(max)
print(min)

186322432
106891806


In [6]:
xkf1_m3_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m3_fault10 = xkf1_m3_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_fault10 = xkf1_m3_fault10[((xkf1_m3_fault10['TimeUS'] >= min) & (xkf1_m3_fault10['TimeUS'] <= max	))]
xkf1_m3_fault10 = xkf1_m3_fault10.reset_index(drop=True)
print(xkf1_m3_fault10)

         TimeUS  Roll  Pitch    Yaw
0     106905542 -0.33   0.35  12.46
1     106918323 -0.39   0.39  12.45
2     106928766 -0.20   0.37  12.46
3     106939536 -0.15   0.37  12.46
4     106950235 -0.17   0.36  12.45
...         ...   ...    ...    ...
7189  186278947 -0.97   1.00   0.99
7190  186289650 -1.03   0.97   1.00
7191  186299874 -1.02   0.92   1.03
7192  186309774 -0.92   0.81   1.03
7193  186319793 -0.76   0.68   1.03

[7194 rows x 4 columns]


In [7]:
att_m3_fault10 = pd.read_csv("ATT.csv")
att_m3_fault10 = att_m3_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_fault10 = att_m3_fault10[((att_m3_fault10['TimeUS'] >= min) & (att_m3_fault10['TimeUS'] <= max	))]
att_m3_fault10=att_m3_fault10.reset_index(drop=True)
print(att_m3_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     106906700     0.04 -0.33     -0.01   0.35   12.39  12.46
1     106919694     0.02 -0.39      0.00   0.39   12.39  12.45
2     106930531     0.00 -0.20      0.00   0.37   12.39  12.46
3     106941454     0.00 -0.15      0.00   0.37   12.39  12.46
4     106951490    -0.02 -0.17      0.00   0.36   12.39  12.45
...         ...      ...   ...       ...    ...     ...    ...
7189  186280650     1.31 -0.97     -0.67   1.00    4.83   0.99
7190  186291048     1.36 -1.03     -0.64   0.97    4.83   1.00
7191  186301226     1.39 -1.02     -0.59   0.92    4.83   1.03
7192  186311233     1.38 -0.92     -0.52   0.81    4.83   1.03
7193  186321744     1.33 -0.76     -0.43   0.68    4.83   1.03

[7194 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m3_fault10)>len(xkf1_m3_fault10):
    to_add=att_m3_fault10[len(xkf1_m3_fault10):]
    att_m3_fault10=att_m3_fault10[:len(xkf1_m3_fault10)]

if len(xkf1_m3_fault10)>len(att_m3_fault10):
    to_add=xkf1_m3_fault10[len(att_m3_fault10):]

for idx,i in enumerate(att_m3_fault10['Roll']):
    if(xkf1_m3_fault10['Roll'][idx] != i):
        att_m3_fault10['Roll'][idx] = ((att_m3_fault10['Roll'][idx] + xkf1_m3_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m3_fault10['Pitch']):
    if(xkf1_m3_fault10['Pitch'][idx] != i):
        att_m3_fault10['Pitch'][idx] = ((att_m3_fault10['Pitch'][idx] + xkf1_m3_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_fault10['Yaw']):
    if(xkf1_m3_fault10['Yaw'][idx] != i):
        att_m3_fault10['Yaw'][idx] = ((att_m3_fault10['Yaw'][idx] + xkf1_m3_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m3_fault10=pd.concat([att_m3_fault10,to_add])

print(att_m3_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     106906700     0.04 -0.33     -0.01   0.35   12.39  12.46
1     106919694     0.02 -0.39      0.00   0.39   12.39  12.45
2     106930531     0.00 -0.20      0.00   0.37   12.39  12.46
3     106941454     0.00 -0.15      0.00   0.37   12.39  12.46
4     106951490    -0.02 -0.17      0.00   0.36   12.39  12.45
...         ...      ...   ...       ...    ...     ...    ...
7189  186280650     1.31 -0.97     -0.67   1.00    4.83   0.99
7190  186291048     1.36 -1.03     -0.64   0.97    4.83   1.00
7191  186301226     1.39 -1.02     -0.59   0.92    4.83   1.03
7192  186311233     1.38 -0.92     -0.52   0.81    4.83   1.03
7193  186321744     1.33 -0.76     -0.43   0.68    4.83   1.03

[7194 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m3_fault10 = esc_0_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_fault10 = esc_0_m3_fault10[((esc_0_m3_fault10['TimeUS'] >= min) & (esc_0_m3_fault10['TimeUS'] <= max))]
esc_0_m3_fault10=esc_0_m3_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m3_fault10 = esc_1_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_fault10 = esc_1_m3_fault10[((esc_1_m3_fault10['TimeUS'] >= min) & (esc_1_m3_fault10['TimeUS'] <= max))]
esc_1_m3_fault10=esc_1_m3_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m3_fault10 = esc_2_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_fault10 = esc_2_m3_fault10[((esc_2_m3_fault10['TimeUS'] >= min) & (esc_2_m3_fault10['TimeUS'] <= max))]
esc_2_m3_fault10=esc_2_m3_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m3_fault10 = esc_3_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_fault10 = esc_3_m3_fault10[((esc_3_m3_fault10['TimeUS'] >= min) & (esc_3_m3_fault10['TimeUS'] <= max))]
esc_3_m3_fault10=esc_3_m3_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m3_fault10 = esc_4_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_fault10 = esc_4_m3_fault10[((esc_4_m3_fault10['TimeUS'] >= min) & (esc_4_m3_fault10['TimeUS'] <= max))]
esc_4_m3_fault10=esc_4_m3_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m3_fault10 = esc_5_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_fault10 = esc_5_m3_fault10[((esc_5_m3_fault10['TimeUS'] >= min) & (esc_5_m3_fault10['TimeUS'] <= max))]
esc_5_m3_fault10=esc_5_m3_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m3_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m3_fault10 = imu_0_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_fault10 = imu_0_m3_fault10[((imu_0_m3_fault10['TimeUS'] >= min) & (imu_0_m3_fault10['TimeUS'] <= max))]

imu_1_m3_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m3_fault10 = imu_1_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_fault10 = imu_1_m3_fault10[((imu_1_m3_fault10['TimeUS'] >= min) & (imu_1_m3_fault10['TimeUS'] <= max))]

imu_2_m3_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m3_fault10 = imu_2_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_fault10 = imu_2_m3_fault10[((imu_2_m3_fault10['TimeUS'] >= min) & (imu_2_m3_fault10['TimeUS'] <= max))]

imu_m3_fault10 = pd.concat((imu_0_m3_fault10, imu_1_m3_fault10, imu_2_m3_fault10))
imu_m3_fault10=imu_m3_fault10.groupby(imu_m3_fault10.TimeUS, as_index=False).mean()

display(imu_m3_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,106893398,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562
1,106896186,-0.205686,-0.056071,0.012061,0.153629,0.735929,-9.858738
2,106898723,-0.079848,0.040186,0.001080,0.093860,0.851389,-9.918224
3,106902129,0.180783,0.050828,-0.016770,0.214891,1.032160,-9.818566
4,106904697,0.305814,-0.087739,-0.038796,0.212236,0.727475,-9.630442
...,...,...,...,...,...,...,...
27341,186311070,0.089517,-0.180582,0.027792,0.618502,1.844759,-12.362567
27342,186313759,0.151039,-0.174859,-0.009015,0.656361,2.363123,-12.738627
27343,186316447,0.319917,-0.200447,-0.010393,0.736293,2.474347,-12.872283
27344,186318718,0.269170,-0.224830,-0.001907,0.871600,2.249914,-12.945027


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m3_fault10 = pd.merge_ordered(imu_m3_fault10,att_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_0_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_1_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_2_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_3_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_4_m3_fault10)
m3_fault10 = pd.merge_ordered(m3_fault10, esc_5_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")
m3_fault10 = pd.merge_ordered(m3_fault10, rcou_m3_fault10)
m3_fault10=m3_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m3_fault10["TimeUS"] = m3_fault10["TimeUS"] - m3_fault10.iloc[0]["TimeUS"]
m3_fault10["TimeUS"] = m3_fault10["TimeUS"].astype(int)
m3_fault10["Guasto"] = guasto

In [13]:
last_cell = m3_fault10.iloc[-1, m3_fault10.columns.get_loc('TimeUS')]
print(last_cell)

79430626


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
27798  79418886
27799  79421743
27800  79424600
27801  79427457
27802  79430314

[27803 rows x 1 columns]


In [16]:
m3_fault10_final = pd.merge_ordered(m3_fault10,df)
m3_fault10_final=m3_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m3_fault10_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.181833 -0.085428  0.024458  0.351882  0.613464  -9.740562   
1          1592 -0.181833 -0.085428  0.024458  0.351882  0.613464  -9.740562   
2          2857 -0.181833 -0.085428  0.024458  0.351882  0.613464  -9.740562   
3          4380 -0.205686 -0.056071  0.012061  0.153629  0.735929  -9.858738   
4          5714 -0.205686 -0.056071  0.012061  0.153629  0.735929  -9.858738   
...         ...       ...       ...       ...       ...       ...        ...   
73166  79427457  0.269170 -0.224830 -0.001907  0.871600  2.249914 -12.945027   
73167  79429814  0.143658 -0.224081 -0.007129  0.964990  2.309602 -13.230780   
73168  79429938  0.143658 -0.224081 -0.007129  0.964990  2.309602 -13.230780   
73169  79430314  0.143658 -0.224081 -0.007129  0.964990  2.309602 -13.230780   
73170  79430626  0.143658 -0.224081 -0.007129  0.964990  2.309602 -13.230780   

       DesRoll  Roll  DesPitch  ...  Cu

In [17]:
m3_fault10_final = m3_fault10_final[m3_fault10_final.TimeUS.isin(to_be)]

In [18]:
m3_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
2,2857,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
4,5714,-0.205686,-0.056071,0.012061,0.153629,0.735929,-9.858738,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
6,8571,-0.079848,0.040186,0.001080,0.093860,0.851389,-9.918224,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
8,11428,0.180783,0.050828,-0.016770,0.214891,1.032160,-9.818566,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73157,79418886,0.231043,-0.164744,0.048283,0.548167,1.842138,-12.117257,1.39,-1.02,-0.59,...,1.99,5457.0,3.08,1521.0,1451.0,1535.0,1435.0,1493.0,1482.0,2.0
73161,79421743,0.089517,-0.180582,0.027792,0.618502,1.844759,-12.362567,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0
73163,79424600,0.151039,-0.174859,-0.009015,0.656361,2.363123,-12.738627,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0
73166,79427457,0.269170,-0.224830,-0.001907,0.871600,2.249914,-12.945027,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0


In [19]:
from datetime import timedelta
m3_fault10_final=m3_fault10_final.reset_index(drop=True)
m3_fault10_final['TimeUS'] = pd.to_datetime(m3_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
1,00:00:00.002857,-0.181833,-0.085428,0.024458,0.351882,0.613464,-9.740562,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
2,00:00:00.005714,-0.205686,-0.056071,0.012061,0.153629,0.735929,-9.858738,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
3,00:00:00.008571,-0.079848,0.040186,0.001080,0.093860,0.851389,-9.918224,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
4,00:00:00.011428,0.180783,0.050828,-0.016770,0.214891,1.032160,-9.818566,0.04,-0.33,-0.01,...,2.96,4043.0,3.47,1479.0,1406.0,1428.0,1459.0,1460.0,1427.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27798,00:01:19.418886,0.231043,-0.164744,0.048283,0.548167,1.842138,-12.117257,1.39,-1.02,-0.59,...,1.99,5457.0,3.08,1521.0,1451.0,1535.0,1435.0,1493.0,1482.0,2.0
27799,00:01:19.421743,0.089517,-0.180582,0.027792,0.618502,1.844759,-12.362567,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0
27800,00:01:19.424600,0.151039,-0.174859,-0.009015,0.656361,2.363123,-12.738627,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0
27801,00:01:19.427457,0.269170,-0.224830,-0.001907,0.871600,2.249914,-12.945027,1.38,-0.92,-0.52,...,1.99,5457.0,3.08,1502.0,1450.0,1546.0,1397.0,1494.0,1458.0,2.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m3_fault10_final)/350)):
        V1=m3_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_22256\2663447777.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_22256\2663447777.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.011828,0.017444,0.197152,0.132418,88.0,66.0,5.799726,5.493375,-0.000067,...,5982.824795,3.533077,0.043951,-0.813871,3.539274,137.0,136.0,23.574628,8.050607,2
1,1.0,-0.012811,0.017734,0.203216,0.133597,101.0,76.0,5.822283,4.054995,0.000341,...,6122.401286,3.534574,0.044614,-0.792384,3.540861,132.0,131.0,23.730508,8.525771,2
2,2.0,-0.013688,0.017956,0.186585,0.134508,104.0,78.0,5.873542,4.953687,0.000722,...,6248.944428,3.536062,0.045269,-0.774510,3.542440,131.0,130.0,23.875707,8.982221,2
3,3.0,-0.014019,0.017944,0.179399,0.134498,68.0,69.0,7.455189,6.211708,0.001022,...,6224.092569,3.537542,0.045917,-0.759867,3.544008,138.0,137.0,24.009859,9.417951,2
4,4.0,-0.014424,0.017951,0.166537,0.134569,76.0,98.0,6.365653,6.312871,0.001143,...,6090.107021,3.539014,0.046556,-0.748116,3.545567,139.0,138.0,24.132620,9.831075,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74.0,-0.040356,0.020050,-0.265077,0.147075,87.0,116.0,9.183131,7.255800,0.025053,...,5524.714057,3.663247,0.124469,0.090730,3.680157,135.0,130.0,24.591693,3.501164,2
75,75.0,-0.040308,0.020004,-0.258574,0.146906,129.0,97.0,7.540277,7.326633,0.025466,...,5602.670923,3.665141,0.125704,0.064856,3.682209,156.0,152.0,24.417458,3.710769,2
76,76.0,-0.040523,0.019976,-0.258541,0.146873,118.0,89.0,7.707908,5.737633,0.025936,...,5665.907410,3.667026,0.126926,0.039129,3.684251,133.0,129.0,24.229803,4.146097,2
77,77.0,-0.040684,0.019941,-0.256063,0.146796,127.0,95.0,7.893308,5.843537,0.026091,...,5617.983020,3.668902,0.128135,0.013573,3.686282,142.0,139.0,24.029341,4.456246,2


In [23]:
path_file = path_file.replace(r"M3\FAULT_M3_10", "")
os.chdir(path_file)
df_merged.to_csv('m3_fault10.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
